In [ ]:
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import re
import json

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    LSTM,
    Bidirectional,
    Conv1D,
    Dense,
    Flatten,
    Input,
    MaxPooling1D,
    TimeDistributed,
    Dropout
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

### Evaluating

In [ ]:
folder = Path("")
model_name = ""

In [ ]:
# Open the JSON file and read the data back into a dictionary
with open(folder/'class_index.json', 'r') as json_file:
    class_index = json.load(json_file)

In [ ]:
class_index

In [ ]:
with open('processed_labeled_image_data.npy', 'rb') as f:
    x_data = np.load(f)

with open('processed_labeled_image_result.npy', 'rb') as f:
    y_data = np.load(f)

In [ ]:
# with open(folder/'processed_labeled_filtered_normalized_sampled_data.npy', 'rb') as f:
#     x_data = np.load(f)

# with open(folder/'processed_labeled_filtered_normalized_sampled_result.npy', 'rb') as f:
#     y_data = np.load(f)

In [ ]:
y_data_categorical = to_categorical(y_data,num_classes=np.unique(y_data).shape[0])

x_train, x_val, y_train, y_val = train_test_split(x_data,y_data_categorical,
                                                  test_size=0.3,
                                                  random_state=42,
                                                  stratify=y_data_categorical)

x_val, x_test, y_val, y_test = train_test_split(x_val,y_val,
                                                  test_size=0.34,
                                                  random_state=42,
                                                  stratify=y_val)

In [ ]:
# y_data_categorical = to_categorical(y_data,num_classes=np.unique(y_data).shape[0])

# x_train, x_val, y_train, y_val = train_test_split(x_data,y_data_categorical,
#                                                   test_size=0.3,
#                                                   random_state=42,
#                                                   stratify=y_data_categorical)

In [ ]:
model = tf.keras.models.load_model(folder/f'best_{model_name}_model.keras')

# Display the model architecture
model.summary()

In [ ]:
y_pred = model.predict(x_train)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_classes = np.argmax(y_train, axis=1)

# Calculate precision and recall for multi-class classification
macro_precision = precision_score(y_classes, y_pred_classes, average='macro')
macro_recall = recall_score(y_classes, y_pred_classes, average='macro')

print(f"Macro Precision: {macro_precision}")
print(f"Macro Recall: {macro_recall}")

# Calculate precision and recall for multi-class classification
micro_precision = precision_score(y_classes, y_pred_classes, average='micro')
micro_recall = recall_score(y_classes, y_pred_classes, average='micro')

print(f"Micro Precision: {micro_precision}")
print(f"Micro Recall: {micro_recall}")

# Create the confusion matrix
conf_matrix = confusion_matrix(y_classes, y_pred_classes)

# Convert to DataFrame for better readability
conf_matrix_df = pd.DataFrame(conf_matrix, index=class_index.keys(), columns=class_index.keys())

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title(f'Matrix de Confusão - Treino - {model_name}')
plt.savefig(folder/f"confusion_matrix_train_{model_name}.png")
plt.show()

In [ ]:
y_pred = model.predict(x_val)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_classes = np.argmax(y_val, axis=1)

# Calculate precision and recall for multi-class classification
macro_precision = precision_score(y_classes, y_pred_classes, average='macro')
macro_recall = recall_score(y_classes, y_pred_classes, average='macro')

print(f"Macro Precision: {macro_precision}")
print(f"Macro Recall: {macro_recall}")

# Calculate precision and recall for multi-class classification
micro_precision = precision_score(y_classes, y_pred_classes, average='micro')
micro_recall = recall_score(y_classes, y_pred_classes, average='micro')

print(f"Micro Precision: {micro_precision}")
print(f"Micro Recall: {micro_recall}")

with open(folder/f"results_{model_name}.txt", "w+") as f:
    f.write(f"\\newcommand{{\\{model_name.replace("_","")}MacroPrecision}}{{{macro_precision:0.4}}}\n")
    f.write(f"\\newcommand{{\\{model_name.replace("_","")}MacroRecall}}{{{macro_recall:0.4}}}\n")
    f.write(f"\\newcommand{{\\{model_name.replace("_","")}MicroPrecision}}{{{micro_precision:0.4}}}\n")
    f.write(f"\\newcommand{{\\{model_name.replace("_","")}MicroRecall}}{{{micro_recall:0.4}}}\n")

# Create the confusion matrix
conf_matrix = confusion_matrix(y_classes, y_pred_classes)

# Convert to DataFrame for better readability
conf_matrix_df = pd.DataFrame(conf_matrix, index=class_index.keys(), columns=class_index.keys())

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title(f'Matrix de Confusão - Teste - {model_name}')
plt.savefig(folder/f"confusion_matrix_train_{model_name}.png")
plt.show()

#### History

In [ ]:
# Path to the JSON file
file_path = folder/f'training_history_{model_name}.json'

# Read the JSON file
with open(file_path, 'r') as f:
    history = json.load(f)

# Print the training history
print(history.keys())

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history["val_accuracy"], marker='o', linestyle='-', color='navy', label="Acurácia de validação")
plt.plot(history["accuracy"], marker='s', linestyle='-', color='darkorange', label="Acurácia de treino")
plt.axvline(x=np.array(history["val_accuracy"]).argmax(), color='darkred', linestyle='--', label='Best model')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.savefig(folder/f"train_epochs_{model_name}.png")
plt.show()